In [ ]:
!pip install datasets -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers -qq

In [ ]:
!pip install accelerate -qq

In [ ]:
!pip install python-Levenshtein -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install fuzzywuzzy -qq

In [ ]:
from typing import Dict, List, Union

In [ ]:
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [ ]:
MessageHint = Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]

In [ ]:
# def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> MessageHint:
#     """
#     Функция добавляет к текстам prompt с инструкцией для LLM.
#     """
#     assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'

#     categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
#     if isinstance(text, str):
#         prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
#                  f'списка тем наиболее представлена в следующем тексте. ' \
#                  f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
#                  f'Список тем: {categories_as_string}.\nТекст: {" ".join(text.split())}\nВаш ответ: '
#         messages = [
#             {
#                 'role': 'system',
#                 'content':
#                 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
#             },
#             {
#                 'role': 'user',
#                 'content': prompt
#             }
#         ]
#     else:
#         messages = []
#         for it in text:
#             prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
#                      f'списка тем наиболее представлена в следующем тексте. ' \
#                      f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
#                      f'Список тем: {categories_as_string}.\nТекст: {" ".join(text.split())}\nВаш ответ: '
#             messages.append([
#                 {
#                     'role': 'system',
#                     'content':
#                     'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
#                 },
#                 {
#                     'role': 'user',
#                     'content': prompt
#                 }
#             ])
#     return {'message_for_llm': messages}

In [ ]:
# from typing import Union, List, Dict

# def create_prompt(single_text: str, instruction) -> List[Dict[str, str]]:
#     # Подготовка промпта с явным акцентом на строгий выбор из списка
#     prompt = (
#         f"{instruction}\nТекст: {single_text.strip()}\nВаш ответ: "
#     )
#     return [
#         {
#             'role': 'system',
#             'content': (
#                 'Вы – экспертный помощник, который умеет анализировать тексты на русском языке. '
#                 'Вы должны выбрать одну тему из заданного списка, опираясь на суть текста, без самостоятельных выводов.'
#             )
#         },
#         {
#             'role': 'user',
#             'content': prompt
#         }
#     ]

# def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> Dict[str, List[Dict[str, str]]]:
#     """
#     Функция добавляет к текстам prompt с инструкцией для LLM, чтобы модель выбирала ответ строго из предложенного списка тем.
#     """
#     assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'

#     categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
#     instruction = (
#         f"Прочтите следующий текст и строго выберите наиболее подходящую тему из предложенного списка. "
#         f"Ответ должен быть на русском языке. "
#         f"Ответ должен содержать только название темы, без дополнительных слов, фраз или интерпретаций. "
#         f"Ваш ответ должен быть одним из следующих вариантов: {categories_as_string}."
#     )

#     if isinstance(text, str):
#         messages = create_prompt(text, instruction)
#     else:
#         messages = [create_prompt(t, instruction) for t in text]

#     return {'message_for_llm': messages}

In [ ]:
from typing import Union, List, Dict

def create_prompt(single_text: str, instruction) -> List[Dict[str, str]]:
    # Подготовка промпта с явным акцентом на строгий выбор из списка
    prompt = (
        f"{instruction}\nТекст: {single_text.strip()}\nВаш ответ: "
    )
    return [
        {
            'role': 'system',
            'content': (
                'Вы – экспертный помощник, который умеет анализировать тексты на русском языке. '
                'Вы должны выбрать одну тему из заданного списка, опираясь на суть текста, без самостоятельных выводов.'
            )
        },
        {
            'role': 'user',
            'content': prompt
        }
    ]

def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> Dict[str, List[Dict[str, str]]]:
    """
    Функция добавляет к текстам prompt с инструкцией для LLM, чтобы модель выбирала ответ строго из предложенного списка тем.
    """
    assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'

    categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
    instruction = (
        f"Прочтите следующий текст и строго выберите наиболее подходящую тему из предложенного списка. "
        f"Ответ должен быть на русском языке. "
        f"Ответ должен содержать только название темы, без дополнительных слов, фраз или интерпретаций. "
        f"Ваш ответ должен быть одним из следующих вариантов: {categories_as_string}."
    )

    if isinstance(text, str):
        messages = create_prompt(text, instruction)
    else:
        messages = [create_prompt(t, instruction) for t in text]

    return {'message_for_llm': messages}

In [ ]:
test_message = """Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).
Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся с
неразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенно
различными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».
В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием мира.
""".replace('\n', '')
test_categories = ['философия', 'математика', 'физика']

prepare_message_for_llm(test_message, test_categories)

{'message_for_llm': [{'role': 'system',
   'content': 'Вы – экспертный помощник, который умеет анализировать тексты на русском языке. Вы должны выбрать одну тему из заданного списка, опираясь на суть текста, без самостоятельных выводов.'},
  {'role': 'user',
   'content': 'Прочтите следующий текст и строго выберите наиболее подходящую тему из предложенного списка. Ответ должен быть на русском языке. Ответ должен содержать только название темы, без дополнительных слов, фраз или интерпретаций. Ваш ответ должен быть одним из следующих вариантов: философия, математика и физика.\nТекст: Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся снеразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенноразличными самостоятельными субстанциями и не могут, следовательно, иметь ничего общег

# Qwen2
Technical report - https://arxiv.org/pdf/2407.10671  
https://qwenlm.github.io/blog/qwen2/


* Размеры от 0.5B до 72B.
* 29 языков, включая русский.
* Поддержка контекстного окна до 128K токенов у Qwen2-7B-Instruct и Qwen2-72B-Instruct.
* Использование Grouped Query Attention (GQA).



In [ ]:
llm_pipeline = pipeline(model='Qwen/Qwen2-1.5B-Instruct', device_map='auto', torch_dtype='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

### Выгрузим данные
https://huggingface.co/datasets/Davlan/sib200/viewer/rus_Cyrl

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


In [ ]:
validation_set[:1]

{'index_id': [548],
 'category': ['sports'],
 'text': ['Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.']}

### Добавим prompt

In [ ]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], list_of_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], list_of_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [ ]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [ ]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы – экспертный помощник, который умеет анализировать тексты на русском языке. Вы должны выбрать одну тему из заданного списка, опираясь на суть текста, без самостоятельных выводов.', 'role': 'system'}, {'content': 'Прочтите следующий текст и строго выберите наиболее подходящую тему из предложенного списка. Ответ должен быть на русском языке. Ответ должен содержать только название темы, без дополнительных слов, фраз или интерпретаций. Ваш ответ должен быть одним из следующих вариантов: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.\nВаш ответ: ', 'role': 'user'}]


### Сгенерируем ответы модели

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(y_pred[0])

[{'content': 'Вы – экспертный помощник, который умеет анализировать тексты на русском языке. Вы должны выбрать одну тему из заданного списка, опираясь на суть текста, без самостоятельных выводов.', 'role': 'system'}, {'content': 'Прочтите следующий текст и строго выберите наиболее подходящую тему из предложенного списка. Ответ должен быть на русском языке. Ответ должен содержать только название темы, без дополнительных слов, фраз или интерпретаций. Ваш ответ должен быть одним из следующих вариантов: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.\nВаш ответ: ', 'role': 'user'}, {'role': 'assistant', 'content': 'sport'}]


### Результаты

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                        precision    recall  f1-score   support

               biology       0.00      0.00      0.00         0
               cooking       0.00      0.00      0.00         0
         entertainment       1.00      0.44      0.62         9
              gambling       0.00      0.00      0.00         0
             geography       0.26      0.62      0.37         8
                health       1.00      0.45      0.62        11
              politics       0.92      0.79      0.85        14
    science/technology       0.79      0.76      0.78        25
                snores       0.00      0.00      0.00         0
                 sport       0.00      0.00      0.00         0
                sports       0.57      0.33      0.42        12
        transportation       0.00      0.00      0.00         0
                travel       1.00      0.25      0.40        20
               weather       0.00      0.00      0.00         0
Заболевание, переносим       0.00      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
len(set([x[-1]['content'] for x in y_pred]))

16

In [ ]:
print(set([x[-1]['content'] for x in y_pred]))
print(list_of_categories)
print(set([x[-1]['content'] for x in y_pred]) - set(list_of_categories))
print(set(list_of_categories) - set([x[-1]['content'] for x in y_pred]))

{'cooking', 'Заболевание, переносим', 'weather', 'snores', 'biology', 'entertainment', 'sports', 'geography', 'transportation', 'sport', 'travel', 'politics', 'gambling', 'health', 'science/technology', '交通拥堵'}
['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']
{'cooking', 'weather', 'snores', 'biology', 'transportation', 'sport', 'gambling', 'Заболевание, переносим', '交通拥堵'}
set()


**Модель сегенрировала лишние ответы**

In [ ]:
llm_pipeline(prepare_message_for_llm(test_message, test_categories)['message_for_llm'], max_new_tokens=500)

[{'generated_text': [{'role': 'system',
    'content': 'Вы – экспертный помощник, который умеет анализировать тексты на русском языке. Вы должны выбрать одну тему из заданного списка, опираясь на суть текста, без самостоятельных выводов.'},
   {'role': 'user',
    'content': 'Прочтите следующий текст и строго выберите наиболее подходящую тему из предложенного списка. Ответ должен быть на русском языке. Ответ должен содержать только название темы, без дополнительных слов, фраз или интерпретаций. Ваш ответ должен быть одним из следующих вариантов: философия, математика и физика.\nТекст: Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся снеразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенноразличными самостоятельными субстанциями и не могут, следовательно, иметь ничего об

### Применим постобработку ответов модели
Воспользуемся расстоянием Левенштейна

In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    tqdm(y_pred)
))

  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
print(y_pred_with_normalization[0])

sports


In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.83      0.56      0.67         9
         geography       0.26      0.62      0.37         8
            health       0.83      0.45      0.59        11
          politics       0.92      0.79      0.85        14
science/technology       0.74      0.80      0.77        25
            sports       0.50      1.00      0.67        12
            travel       1.00      0.25      0.40        20

          accuracy                           0.64        99
         macro avg       0.73      0.64      0.62        99
      weighted avg       0.77      0.64      0.63        99



## 1.5B
                    precision    recall  f1-score   support

     entertainment       0.50      0.11      0.18         9
         geography       0.38      0.38      0.38         8
            health       0.86      0.55      0.67        11
          politics       1.00      0.57      0.73        14  
science/technology       0.59      0.96      0.73        25  
            sports       0.48      0.83      0.61        12  
            travel       0.75      0.45      0.56        20

          accuracy                           0.62        99
         macro avg       0.65      0.55      !0.55        99
      weighted avg       0.67      0.62      !0.59        99

 ## 7B
                    precision    recall  f1-score   support

     entertainment       0.57      0.44      0.50         9
         geography       0.50      0.50      0.50         8
            health       0.80      0.36      0.50        11
          politics       0.76      0.93      0.84        14  
science/technology       0.72      0.92      0.81        25  
            sports       0.79      0.92      0.85        12  
            travel       0.69      0.55      0.61        20

          accuracy                           0.71        99
         macro avg       0.69      0.66      !0.66        99
      weighted avg       0.71      0.71      !0.69        99

### Посмотрим результаты на тестовом подмножестве

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

            Health       0.00      0.00      0.00         0
            Travel       0.00      0.00      0.00         0
           biology       0.00      0.00      0.00         0
         chemistry       0.00      0.00      0.00         0
             drugs       0.00      0.00      0.00         0
     entertainment       0.60      0.16      0.25        19
          genetics       0.00      0.00      0.00         0
      geo-politics       0.00      0.00      0.00         0
         geography       0.48      0.76      0.59        17
            health       0.50      0.36      0.42        22
             music       0.00      0.00      0.00         0
          politics       0.89      0.57      0.69        30
          religion       0.00      0.00      0.00         0
science/technology       0.66      0.69      0.67        51
          security       0.00      0.00      0.00         0
  sound production       0.00      0.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Также применим постобработку

In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.44      0.21      0.29        19
         geography       0.48      0.76      0.59        17
            health       0.50      0.45      0.48        22
          politics       0.78      0.60      0.68        30
science/technology       0.67      0.71      0.69        51
            sports       0.40      0.88      0.55        25
            travel       0.94      0.38      0.54        40

          accuracy                           0.58       204
         macro avg       0.60      0.57      0.54       204
      weighted avg       0.65      0.58      0.57       204



## 1.5B
                    precision    recall  f1-score   support

     entertainment       0.00      0.00      0.00        19
         geography       0.60      0.35      0.44        17
            health       0.75      0.41      0.53        22
          politics       0.90      0.60      0.72        30  
science/technology       0.45      0.96      0.61        51  
            sports       0.57      0.84      0.68        25  
            travel       0.88      0.35      0.50        40

          accuracy                           0.57       204
         macro avg       0.59      0.50      !0.50       204
      weighted avg       0.62      0.57      !0.53       204

 ## 7B

                      precision    recall  f1-score   support
  
       entertainment       0.83      0.53      0.65        19
           geography       0.80      0.71      0.75        17
              health       0.82      0.82      0.82        22
            politics       0.66      0.97      0.78        30  
  science/technology       0.81      0.94      0.87        51  
              sports       0.87      0.80      0.83        25  
              travel       0.93      0.68      0.78        40

            accuracy                           0.80       204
           macro avg       0.82      0.78      !0.78       204
        weighted avg       0.82      0.80      !0.80       204